In [ ]:
%pip install roboflow
%pip install inference-sdk
%pip install torch
%pip install torchvision

%pip install ultralytics==8.0.196
%pip install pyyaml

In [ ]:
import random
import torch
import torchvision
import numpy as np
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import yaml
import torchvision.transforms as T

print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

In [ ]:


from roboflow import Roboflow
rf = Roboflow(api_key="o2Vk0Lj1qpKf8uGx9lZJ")
project = rf.workspace("bolbol-duawh").project("trafic_lights_detection")
version = project.version(3)
dataset = version.download("yolov8")
                

In [ ]:


class TrafficLightDataset(Dataset):
    def __init__(self, root_dir, split, transforms=None):
        self.root_dir = root_dir
        self.split = split  # 'train', 'test', or 'valid'
        self.transforms = transforms
        
        split_dir = os.path.join(root_dir, split)
        self.img_dir = os.path.join(split_dir, "images")
        self.label_dir = os.path.join(split_dir, "labels")
        
        self.imgs = list(sorted(os.listdir(self.img_dir)))
        self.labels = list(sorted(os.listdir(self.label_dir)))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.imgs[idx])
        label_path = os.path.join(self.label_dir, self.labels[idx])
        
        img = Image.open(img_path).convert("RGB")
        
        target = self.parse_label(label_path)
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        # Ensure boxes is always 2D
        if target['boxes'].ndim == 1:
            target['boxes'] = target['boxes'].unsqueeze(0)
        
        # Ensure labels is always 1D
        target['labels'] = target['labels'].view(-1)
        
        return img, target

    def parse_label(self, label_path):
        boxes = []
        labels = []
        with open(label_path, 'r') as f:
            for line in f:
                values = line.strip().split()
                class_id = int(values[0])
                labels.append(class_id)
                x_center, y_center, width, height = map(float, values[1:])
                x_min = x_center - width / 2
                y_min = y_center - height / 2
                x_max = x_center + width / 2
                y_max = y_center + height / 2
                boxes.append([x_min, y_min, x_max, y_max])
        
        # Handle empty labels
        if not boxes:
            boxes.append([0, 0, 1, 1])  # Add a dummy box
            labels.append(0)  # Add a dummy label (background)
        
        return {
            'boxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64)
        }

print("Class Traffic Light Dataset completed successfully")

In [ ]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            image = F.hflip(image)
            bbox = target["boxes"]
            bbox[:, [0, 2]] = 1 - bbox[:, [2, 0]]
            target["boxes"] = bbox
        return image, target

def get_transform(train):
    transforms = []
    transforms.append(ToTensor())
    if train:
        transforms.append(RandomHorizontalFlip(0.5))
    return Compose(transforms)

print('Class Compose completed successfully')

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights

def get_model(num_classes):
    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    model = fasterrcnn_resnet50_fpn(weights=weights)
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    print(f"Model configured for {num_classes} classes")  # Add this line
    
    return model

def train_model(model, data_loader, optimizer, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        num_batches = 0
        
        for images, targets in data_loader:
            try:
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                
                optimizer.zero_grad()
                losses.backward()
                optimizer.step()
                
                total_loss += losses.item()
                num_batches += 1
            except Exception as e:
                print(f"Error during training: {e}")
                for i, t in enumerate(targets):
                    print(f"Sample {i}:")
                    print(f"  Boxes shape: {t['boxes'].shape}")
                    print(f"  Labels shape: {t['labels'].shape}")
                continue
        
        if num_batches > 0:
            avg_loss = total_loss / num_batches
            print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
        else:
            print(f"Epoch {epoch+1}/{num_epochs}, No valid batches")

print("model training completed successfully")

In [ ]:
def main():
    try:
        with open('./trafic_lights_detection-3/data.yaml', 'r') as file:
            config = yaml.safe_load(file)
        
        print(f"Number of classes in config: {config['nc']}")  # Add this line
        print(f"Class names: {config['names']}")

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")

        # Create datasets for train, validation, and test
        train_dataset = TrafficLightDataset(root_dir='./trafic_lights_detection-3', split='train', transforms=get_transform(train=True))
        val_dataset = TrafficLightDataset(root_dir='./trafic_lights_detection-3', split='valid', transforms=get_transform(train=False))
        test_dataset = TrafficLightDataset(root_dir='./trafic_lights_detection-3', split='test', transforms=get_transform(train=False))

        # Create dataloaders
        train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
        val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
        test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

        # Create model
        num_classes = config['nc'] + 1  # Add 1 for the background class
        model = get_model(num_classes=num_classes)
        model.to(device)

        # Optimizer
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

        # Train
        num_epochs = 10  # You can adjust this or make it a parameter
        try:
            train_model(model, train_loader, optimizer, device, num_epochs=num_epochs)
        except Exception as e:
            print(f"An error occurred during training: {e}")

        # Save model
        torch.save(model.state_dict(), 'traffic_light_model.pth')
        print("Training completed and model saved successfully.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Please check if the data.yaml file exists in the specified location.")
    except Exception as e:
        print(f"An error occurred: {e}")
        
if __name__ == "__main__":
    main()

    print("main ran succesffuly, model has been trained")